# Generate index components using dedicated service - C#

### Overview
Reference data encompasses a wide range of specification data about:
* financial instrument such as asset class, symbol, maturity, etc.
* counterparties such as issuer information.
* pricing such as Open, high low and close prices.

The challenge with reference data is that it tends to be sourced from multiple sources: internal, counterparties and providers.

Reference data dedicated service is a result of various data sources and fields sourcing, validation, cross-checking and normalization.<br>

This sample requests **on-demand** exchange data and enables fields mapping by calling a dedicated static data service, making available clean data.

### Inputs/outputs
Index sample is designed to help you searching and requesting index components reference data by using a wide range of commonly used fields.

It takes an index code as per input filter and returns the *entries* matching the request.

### Services used
This sample uses *gRPC requests* in order to retrieve index components reference data from the hosted service. The queried endpoint in this script are:
* *StaticDataService*: to directly retrieve reference data objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.StaticData.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*

***

# Run Index components sample

### Step 1: Install packages

In [1]:
#i "nuget: file://~/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 0.*-pre*"
#r "nuget: CsvHelper"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Restore sources file://~/.nuget/packages/ https://api.nuget.org/v3/index.json Installed Packages CsvHelper, 29.0.0 Systemathics.Apis, 0.13.2-pre

In [2]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.StaticData.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
// Import/export csv files
using CsvHelper;
using System.IO;
using System.Globalization;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [3]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Request creation

The following code snippet enables to select the **index** by its *Name/code*:

In [4]:
// Set index and exchange codes
var index = "Nasdaq 100";
var exchange = "XNGS";

The following code snippets call the service, generate the request and return the reply: 

In [5]:
// Instantiate the service
var service = new StaticDataService.StaticDataServiceClient(channel);

// Generate request
var request = new StaticDataRequest 
{ 
    AssetType = AssetType.Equity, 
    Index = index, 
    Count = 1000 // By default the count is set to 100, make sure to retrieve all the components
};

// Call the service
var reply = service.StaticData(request, headers);

// Display the results
display(reply.Equities.Count)

1000

### Step 4: Retrieve data

#### 4.1 Retrieve instruments' identifiers (mapping)

In [6]:
// Create a class to handle the instrument's mapping codes
public class Instrument
{
    public string Ticker{get; set;}
    public string Exchange{get; set;}
    public string Name{get; set;}
    public string Isin{get; set;}
    public string Cusip{get; set;}
    public string Sedol{get; set;}
    public string Bloomberg{get; set;}
}
var instruments = new List<Instrument>();

// Get mapping codes for the requested instruments
foreach (var reply in reply.Equities) 
{
    // Get mapping codes
    var ticker = reply.Identifier.Ticker;
    reply.Mapping.TryGetValue("Bloomberg", out var bloomberg);
    
    // Add the instruments
    if(reply.Identifier.Exchange == exchange)
    instruments.Add(new Instrument()
    {
        Ticker = ticker, Exchange = reply.Identifier.Exchange, Name = reply.Name, Isin = reply.Isin, Cusip = reply.Cusip, Sedol = reply.Sedol, Bloomberg = bloomberg
    });
}

In [7]:
display(instruments.Count)

306

The following code snippet exports data to *csv* file:

In [8]:
var d = new DirectoryInfo("output"); if (!d.Exists) { d.Create(); }
using (var writer = new StreamWriter($"output/{index.ToLowerInvariant()} components.csv"))
using (var csv = new CsvWriter(writer, CultureInfo.InvariantCulture))
{
    csv.WriteRecords(instruments);
}